<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Rodrigo Guerra
- Nombre de alumno 2: Mariano Suárez


### **Link de repositorio de GitHub:** `https://github.com/MarianoSuarez03/Laboratorios-Python`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 12/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [ ]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
df_retail['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

In [ ]:
df_retail['Recency'] = (df_retail['InvoiceDate'].max() - df_retail['InvoiceDate']).dt.days()

TypeError: 'Series' object is not callable

## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
# LRMFP = FunctionTransformer(custom_features)

def custom_features(dataframe_in):
    # Length
    df_length = dataframe_in.copy()
    df_length = (
        df_length.groupby("Customer ID")["InvoiceDate"]
        .agg(["first", "last"])
        .reset_index()
    )
    df_length["Length"] = (df_length["last"] - df_length["first"]).dt.days
    df_length = df_length.loc[:, ["Customer ID", "Length"]]
    df_out = df_length

    # Recency
    df_aux = (
        dataframe_in.copy()
        .groupby("Customer ID")
        .max()
        .reset_index()
        .rename(columns={"InvoiceDate": "Max"})
        .loc[:, ["Customer ID", "Max"]]
    )
    df_recency = dataframe_in.merge(df_aux, how='left')
    df_recency["Recency"] = (df_recency["Max"] + datetime.timedelta(days = 1) - df_recency["InvoiceDate"]).dt.days
    df_recency = (
        df_recency.groupby("Customer ID")["Recency"].mean().reset_index().round()
    )
    df_out = pd.merge(df_out, df_recency, how='left')

    # Frequency
    df_frequency = dataframe_in.copy()
    df_frequency = df_frequency.groupby('Invoice').first().reset_index()
    df_frequency = df_frequency.groupby('Customer ID').size().reset_index().rename({0: 'Frequency'},axis='columns')
    df_out = pd.merge(df_out, df_frequency, how='left')

    #Monetary
    
    df_monetary = dataframe_in.copy()
    df_monetary['Monetary'] = df_monetary['Quantity']*df_monetary['Price']
    df_monetary = df_monetary.groupby('Customer ID').sum().loc[:,'Monetary'].reset_index()
    df_out = pd.merge(df_out, df_monetary, how='left')
    df_out['Monetary'] = df_out['Monetary']/df_out['Frequency']
    
    # Periodicity
    df_periodicity = dataframe_in.copy()
    df_periodicity = df_periodicity.groupby('Invoice').first().reset_index() 
    df_periodicity = df_periodicity.sort_values(['Customer ID', 'InvoiceDate'])
    df_periodicity['PrevDate'] = df_periodicity.groupby('Customer ID')['InvoiceDate'].shift()
    df_periodicity['Periodicity'] = (df_periodicity['InvoiceDate'] - df_periodicity['PrevDate']).dt.seconds/(3600*24)
    df_periodicity = df_periodicity.groupby('Customer ID').std().loc[:,['Periodicity']].reset_index()
    
    df_out = pd.merge(df_out, df_periodicity, how='left')
    df_out['Customer ID'] = df_out['Customer ID'].astype(int)

    df_out = df_out.dropna()

    return df_out.set_index('Customer ID')

    


df_custom = custom_features(df_retail)
display(df_custom)


# df_LRMFP = LRMFP.fit_transform(df_retail)


,Length,Recency,Frequency,Monetary,Periodicity
Customer ID,,,,,
12346,196,69.0,11,33.896364,0.477915
12349,181,84.0,3,890.380000,0.053524
12356,44,24.0,3,1186.766667,0.510688
12358,356,159.0,3,906.336667,0.529839
12359,308,167.0,6,427.226667,0.445477
...,...,...,...,...,...
18272,97,57.0,3,435.800000,0.021606
18276,336,148.0,5,264.132000,0.414071
18277,97,39.0,4,267.417500,0.078341


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = (X - X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        return X

# MinMax().transform(ola)

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [ ]:

funcionFeatures = FunctionTransformer(custom_features)
MinMax1 = ColumnTransformer(MinMax)
LRMFP = Pipeline([('Preprocessing',funcionFeatures), 
                ('MinMaxTransform', MinMax()),
                ('T-SNE', TSNE())])

#LRMFP = FunctionTransformer(custom_features)
df_LRMFP = LRMFP.fit_transform(df_retail)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
df_LRMFP

array([[ 20.565302 ,  20.080679 ],
       [-42.699104 ,  35.328327 ],
       [ 34.905228 ,  44.506115 ],
       ...,
       [-51.396347 ,  21.276516 ],
       [  9.055221 , -18.348946 ],
       [ 25.782877 ,  12.0999975]], dtype=float32)

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [ ]:
intertias = [
    [i, KMeans(n_clusters=i, random_state=0).fit(df_LRMFP).inertia_]
    for i in range(2, 20)
]
intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)
px.line(intertias, x="n° clusters", y="inertia", title="Método del Codo con K-Means")

A partir del gráfico, se observa que hay una mayor caída en la disminución del error cuadrático medio a partir de los 4 clusters, pero esta aún mantiene una pendiente considerable, por lo que se decide utilizar 6 clusters, ya que desde ese punto el error comienza a bajar con una pendiente más cercana a 0.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [ ]:
#KMeans(n_clusters=6, random_state=0).fit(df_LRMFP).
kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(df_LRMFP)
df_custom['Cluster'] = kmeans.labels_
df_LRMFP= pd.DataFrame(df_LRMFP)
df_LRMFP['Cluster'] = kmeans.labels_


/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.



In [ ]:
df_custom = df_custom.groupby('Cluster').mean()
df_custom

,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,196.105386,97.318501,5.021077,376.591813,0.485901
1,352.694444,171.123016,15.170635,441.322728,0.419441
2,159.034351,82.904580,3.297710,361.276212,0.047716
3,278.744604,115.039568,7.381295,386.943680,0.411317
4,87.893082,45.694969,4.801887,395.816114,0.472032
5,284.285714,165.965986,6.602041,422.741161,0.479416


Dado el clustering realizado, y a través las medias de _LRFMP_ para los grupos, se pueden observar agrupaciones coherentes, a simple vista, el **cluster 2** corresponde a aquellos clientes que tienen una _Periodicity_ promedio significativamente mas baja que los demás grupos, es decir son clientes que visitaron con frecuencia la tienda, sin embargo, se puede apreciar también que este mismo grupo tiene una _Frequency_ relativamente baja, realizando en promedio 3.3 visitas durante el período estudiado. Por otro lado, se puede apreciar a **cluster 1** que posee los clientes mas fieles, es decir con un mayor _Length_, además de ser los que más gastan en promedio por visita (_Monetary_), además de visitar más seguido(_Frequency_) mientras que el **cluster 4** son aquellos clientes que comenzaron a comprar recientemente en el retail de Mr. Lepin, en comparación con otros grupos, además de tener una tendencia a repetir la compra en un periodo más acotado, reflejado en su bajo _Recency_.

Los **grupos 0, 3 y 5** son similares entre ellos, con la diferencia en que este último tiene clientes con una mayor _Recency_ que los grupos 0 y 3, además de gastar mas dinero en promedio y visitar mas frecuentemente. Por otro lado, el **grupo 0** posee los clientes menos fieles de estos tres grupos, con una menor _Recency_ y una mayor _Periodicty_, mientras que el **grupo 3** tiene aquellos clientes que visitan mas frecuentemente.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [ ]:
df_LRMFP

,0,1,Cluster
0,20.565302,20.080679,0
1,-42.699104,35.328327,2
2,34.905228,44.506115,4
3,1.694678,-44.368725,1
4,-0.377237,-25.104393,5
...,...,...,...
2078,-52.336926,23.623056,2
2079,-21.654217,-32.342800,1
2080,-51.396347,21.276516,2
2081,9.055221,-18.348946,5


In [ ]:
fig = px.scatter(df_LRMFP, x=0, y=1, color= 'Cluster' )
fig.show()

Como se puede apreciar, la representación visual de los clusters corrobora el análisis realizado en la sección 1.3.3.2, dado que los clusters 1, 2 y 4 son aquellos que tienen las medias mas extremas de todos los grupos, ubicandose en los extremos de la figura, en particular el cluster 2 son aquellos clientes que visitaron menos veces, pero con un intervalo de tiempo acotado entre visitas.

Por otro lado, y en línea con lo explicado anteriormente, los clusters 0, 3 y 5 son similares entre ellos, diferenciados por leves cambios en las medias de LRFMP al analizar estos 3 grupos.

Dicho esto, se puede apreciar una distinción clara de los grupos, al tener en cuenta las conclusiones de la sección anterior.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1d355b97-bbb9-4f56-823f-44a85d475e0a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>